In [1]:
import numpy as np
from sklearn import svm
import dv
import cv2
import time

In [2]:
labels = np.genfromtxt('user03_fluorescent_led_labels.csv', delimiter=',', skip_header=1, dtype=int)

start_time = labels[0][1]
for i in labels:
    i[1] = i[1]-start_time
    i[2] = i[2]-start_time
print(labels)

[[       1        0  5934314]
 [       2  6465218 12932606]
 [       3 13560039 20445715]
 [       4 21571877 29310219]
 [       5 29535452 36220000]
 [       6 37048561 43065484]
 [       7 43387244 49355903]
 [       8 49918984 56000259]
 [       8 56338107 62450000]
 [       9 63014639 68170000]
 [      10 69031562 75370245]
 [      11 75788534 79190000]]


In [3]:
# Return the start time of the given class index.
def start(index):
    return labels[index][1]
 
# Return the end time of the given class index.
def end(index):
    return labels[index][2]

def classID(index):
    return labels[index][0]

# Return the class of the current frame. If no gesture then return -1.
def setGesture(currentTime):
    for i in range(len(labels)-1):
        if (currentTime >= start(i) and currentTime <= end(i)):
            return classID(i)
    return -1

In [4]:
# Test the setGesture function

tests = [[42148585, 0], # Gesture 1 has just started 
        [42148584, -1], # Gesture 1 is about to started
        [48082899, 0],  # Gesture 1 is about to end
        [48082900, -1], # Gesture 1 has just ended
         
        [48613803, 1],  # Gesture 2 just started
        [55708624, 2],
        [63720462, 3],
        [121338586, -1],]

for i in tests:
    if (setGesture(i[0]) == i[1]):
        print("Pass")
    else:
        print(setGesture(i[0]), i[1])

6 0
6 -1
7 0
7 -1
7 1
8 2
9 3
Pass


In [5]:
X = []
Y = []
offset = 0
timestamp = 0

with dv.AedatFile("dvSave-2022_10_10_21_31_36.aedat4") as f: 
    for frame in f['frames']:    
        # Set the offset on the first frame.
        if (offset == 0):
            offset = frame.timestamp

        gesture = setGesture((frame.timestamp - offset))
        image = frame.image
        
        # font
        font = cv2.FONT_HERSHEY_SIMPLEX
        
        # org
        org = (5, 30)

        # fontScale
        fontScale = 1

        # Blue color in BGR
        color = (255, 0, 0)

        # Line thickness of 2 px
        thickness = 2
        image = cv2.putText(image, str(gesture), org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        
        cv2.imshow('out',image)
        cv2.waitKey(1)
        
        if (gesture == -1):
            continue
        else:
            Y.append(gesture)

            matrix = frame.image.flatten()
            X.append(matrix)

In [6]:
print(len(X))
print(len(Y))

2103
2103


In [ ]:
clf = svm.SVC()
clf.fit(X, Y)

In [ ]:
clf.predict([X[1780]])